In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from PIL import Image  # Used for image loading

# ✅ Step 1: Load and Preprocess Dataset Without OpenCV
dataset_path = "/home/student/Desktop/71772217105/ML/dataset"  # Path to dataset folder

images = []
labels = []

# Load images using PIL instead of OpenCV
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
   
    if os.path.isdir(person_folder):
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
           
            try:
                img = Image.open(img_path).convert("RGB")  # Open and convert to RGB
                img = img.resize((64, 64))  # Resize image
                img = np.array(img) / 255.0  # Normalize pixel values
               
                images.append(img)
                labels.append(person_name)
            except Exception as e:
                print(f"Skipping {img_name}: {e}")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode labels (convert names to numeric values)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encoding of labels
labels_encoded = to_categorical(labels_encoded)

print(f"Dataset Loaded: {len(images)} images, {len(np.unique(labels))} classes")

# ✅ Step 2: Build ANN Model
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),  # Flatten images into a vector
    Dense(128, activation='relu'),  # First hidden layer
    Dense(64, activation='relu'),  # Second hidden layer
    Dense(len(np.unique(labels)), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Step 3: Train the Model
model.fit(images, labels_encoded, epochs=20, validation_split=0.2)

# ✅ Step 4: Predict on New Image Without OpenCV
def recognize_person(image_path):
    """Function to predict person from an image file."""
    try:
        img = Image.open(image_path).convert("RGB")  # Open and convert image
        img = img.resize((64, 64))  # Resize
        img = np.array(img) / 255.0  # Normalize
        img = img.reshape(1, 64, 64, 3)  # Reshape for model input

        prediction = model.predict(img)
        predicted_label = np.argmax(prediction)
        person_name = label_encoder.inverse_transform([predicted_label])[0]

        return person_name
    except Exception as e:
        return f"Error: {e}"

# Example usage:
test_image_path = "/home/student/Desktop/71772217105/ML/moni.jpeg"  # Replace with actual test image path
print(f"Predicted Person: {recognize_person(test_image_path)}")



Dataset Loaded: 4 images, 4 classes
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 1.4894 - accuracy: 0.0000e+00 - val_loss: 12.1508 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 36ms/step - loss: 1.6443 - accuracy: 0.3333 - val_loss: 17.6704 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 44ms/step - loss: 4.8549 - accuracy: 0.3333 - val_loss: 13.6361 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 43ms/step - loss: 0.7769 - accuracy: 0.6667 - val_loss: 15.6823 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 38ms/step - loss: 1.3829 - accuracy: 0.6667 - val_loss: 16.5899 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 41ms/step - loss: 1.1238 - accuracy: 0.6667 - val_loss: 15.2790 - val_accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 0s 38ms/step - loss: 0.1571 - accuracy: 1.0000 -